In [44]:
import requests as req
import json
from pymongo import MongoClient
from pprint import pprint

with open('config.json', 'r') as read_file:
    client = json.load(read_file)

db_client = MongoClient('localhost', 27017)

In [31]:
# functions
def scp_query(query: str = '', inst: str = '', city: str = '', country: str = '', start_year: int = 0, end_year: int = 0):
    inst_q, city_q, country_q = '', '', ''
    if inst:
        inst_q = ' AND '.join(inst.lower().split(' '))
    if city:
        city_q = ' AND '.join(city.lower().split(' '))
    if country:
        country_q = ' AND '.join(country.lower().split(' '))
    query = [
        (f'{query}' if query else ''),
        (f'AFFIL({inst_q})' if inst_q else ''),
        (f'AFFILCITY({city_q})' if city_q else ''),
        (f'AFFILCOUNTRY({country_q})' if country_q else ''),
        (f'(PUBYEAR > {start_year - 1})' if start_year else ''),
        (f'(PUBYEAR < {end_year + 1})' if end_year != 0 else ''),
    ]
    query = filter(lambda element: element, query)
    return ' AND '.join(query)

def scp_lookup(client:dict, query: str, cnt, start_cursor: int = 0, count_per_page: int = 25):
    base = 'https://api.elsevier.com/content/search/scopus'
    params = {
        'query': query,
        'apiKey': client['apikey'],
        'insttoken': client['insttoken'],
        'view': 'COMPLETE',
        'start': f'{start_cursor + (cnt * count_per_page)}',
        'count': f'{count_per_page}',
    }
    headers = {'Accept': 'application/json'}
    return req.get(url=base, params=params)

In [ ]:
start_year = 2017
end_year = 2017
access_type = 'ACCESSTYPE(OA)' #open access
access_type = 'ACCESSTYPE(OTHER)'
affil_IDs = [
    'AF-ID("Sharif University of Technology" 60027666)',
    'AF-ID("University of Tehran" 60022927)',
    'AF-ID("Tarbiat Modares University" 60032053)',
    'AF-ID("Amirkabir University of Technology" 60007751)',
    'AF-ID("Shahid Beheshti University" 60032873)',
    'AF-ID("Ferdowsi University of Mashhad" 60001800)',
]
query = scp_query(query=affil_IDs[3], start_year=start_year, end_year=end_year)

res = scp_lookup(client=client, query=query, cnt=0)

# res.json()
titles = [doc['dc:title'] for doc in res.json()['search-results']['entry']]
for t in titles:
    print(t)
    print()

In [37]:
res.json()['search-results']['opensearch:totalResults']

'2424'

In [46]:
#database
db = db_client['amirkabir']
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
result = posts.insert_one(post_data)
print('One post: {0}'.format(result.inserted_id))

One post: 5cc4330b8e36a50c2ce49466
